In [1]:
import sys
import os

# Ajoute le dossier projet_NLP/ dans le path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Maintenant tu peux importer proprement
from Script import multilabel_preprocessing as mp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # Barre de progression pour Jupyter
pd.set_option('display.max_colwidth', None)

In [2]:
os.getcwd()

'/home/onyxia/work/projet_NLP/Notebooks'

In [3]:
if os.path.exists("../data/df_target_VF.pkl"):
    df = pd.read_pickle('../data/df_target_VF.pkl')
    print("fichier df chargé")

if os.path.exists("../data/df_metadata"):
    df_metadata = pd.read_parquet('../data/df_metadata')
    print("fichier df_metadata chargé")

fichier df chargé


In [4]:
MeSH_vides = df[df['target'].apply(lambda x: not any(x))]
print("nombre cas sans code MeSH catégorie 'disease' :", MeSH_vides.shape)

df = df.drop(MeSH_vides.index)

nombre cas sans code MeSH catégorie 'disease' : (0, 12)


In [5]:
df['case_text'].str.len().describe()

count    10051.000000
mean      3502.605114
std       2399.441600
min        145.000000
25%       2004.500000
50%       3012.000000
75%       4343.000000
max      60512.000000
Name: case_text, dtype: float64

In [6]:
#df_sample = df[df['case_text'].str.len() < 5000]
#df_sample.shape

In [7]:
df[df['target'].apply(lambda x: any(x)) == False].shape

(0, 12)

In [8]:
i=5
print(df['case_text'][i])
print(mp.mesh_labels_from_vector(df['target'][i]))

The case reports of patients with brain metastases secondary to NSCLC treated with gefitinib as part of the compassionate-use programme have recently been reported as published case studies, at the World Conference on Lung Cancer 2003, at the American Society of Clinical Oncology 2003 conference and at the 'Iressa' Clinical Experience (ICE) meeting (Awada, ICE abs; Cappuzzo [a & b], ICE abs; Dieriks, ICE abs; Stein, ICE abs; de la Cruz [a & b], ICE abs; Maione, ICE abs; Martinez, ICE abs; Diaz-Canton, ICE abs; van der Kamp, ICE abs; van Zandwijk [b], ICE abs; Petruzelka [a], ICE abs; Azemar, ICE abs; Roggero, ICE abs; Kowalski, ICE abs; Martin-Algarra, ICE abs; Pavlakis, ICE abs). (See appendix for ICE abstracts). Together, these case reports suggest that gefitinib has activity in NSCLC brain metastases. The results of published case series are shown in Table 1 .
Most notable are the cases with a complete remission of brain metastases. In the largest case series that included 27 patien

In [9]:
range_max = df.shape[0]
rand_index = np.random.randint(0, range_max)  # tire un entier aléatoire entre 0 et range_max - 1
Code_MeSH = df.loc[rand_index, "target"]
print(mp.mesh_labels_from_vector(Code_MeSH))

['C08 – respiratory tract diseases', 'C12 – urologic and male genital diseases', 'C23 – pathological conditions, signs and symptoms']


# Gemini

## Model

In [11]:
from dotenv import load_dotenv
import base64
import os
import requests
import json

import google.generativeai as genai
from google.genai import types

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
with open("../GEMINI/Prompt.txt", "r") as file:
    prompt = file.read()

prompt

"Je vais te founir des codes MeSH en entré. Tous les codes MeSH appartiennent à la catégorie C «disease» et je ne te fournis que des codes des catégories de «\xa0niveau 1\xa0» donc que des codes entre C01 et C26.\n\nA partir des codes MeSH que je te fournis ci - dessous, écrit en français un exemple de cas clinique que l’on pourrait retouvé dans la littérature scientique.\nJe veux que ton texte fasse entre 300 et 600 mots. \n\nAttention un cas clinique peut être associé à plusieurs\n\nAvant de commencer je vais te donner des exemple écrit en anglais bien que tu devras écrire tes cas cliniques en francais.\n__________________________________________________________________________________\n\nExemple 1\n\nMesh term : ['C04 – neoplasms',\n 'C08 – respiratory tract diseases',\n 'C12 – urologic and male genital diseases',\n 'C14 – cardiovascular diseases']\n\nCas clinique associé (exepected output) : \nA 42-year-old, non-smoking, virgin woman presented to our institution with complaints of 

In [13]:
# Load environment variables from the .env file
load_dotenv()

# Set the API key and model information
API_KEY = os.getenv("GEMINI_API_KEY")
# API_URL = 'https://generativelanguage.googleapis.com/v1beta/openai'
# MODEL = "gemini-2.0-flash"

# Check if API key is available
if not API_KEY:
    print("⚠️ Warning: API_KEY not found in .env file. Please add your API key to continue.")

In [14]:
import base64
import os
from google import genai
from google.genai import types


def generate(input_text):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
        types.Content(
            role="model",
            parts=[
                types.Part.from_text(text="""{
  \"Cas_clinique\": \"Mme Dubois, une femme de 68 ans avec des antécédents d'hypertension artérielle et de diabète de type 2, s'est présentée aux urgences en se plaignant d'une douleur thoracique intense et soudaine, irradiant vers son bras gauche, associée à un essoufflement. Elle a décrit la douleur comme une sensation d'oppression, comme si un poids lourd était posé sur sa poitrine. Elle a également rapporté des nausées et des sueurs froides. \\n\\nÀ son arrivée, elle était pâle, anxieuse et tachypnéique. Sa tension artérielle était de 90/60 mmHg, sa fréquence cardiaque de 110 battements par minute et sa saturation en oxygène de 92 % sous air ambiant. L'auscultation pulmonaire révélait des râles crépitants bilatéraux. L'électrocardiogramme (ECG) montrait un sus-décalage du segment ST dans les dérivations antérieures (V1 à V4), suggérant un infarctus du myocarde avec élévation du segment ST (STEMI) antérieur étendu. \\n\\nCompte tenu de la présentation clinique et des résultats de l'ECG, une suspicion forte d'occlusion aiguë de l'artère coronaire antérieure descendante (IVA) a été soulevée. L'équipe médicale a immédiatement initié un traitement comprenant de l'oxygène, de l'aspirine, du clopidogrel et de la morphine pour soulager la douleur. Une thrombolyse intraveineuse a été envisagée, mais étant donné le délai de plus de 2 heures depuis le début des symptômes et la disponibilité d'un centre de cardiologie interventionnelle à proximité, une angioplastie coronaire primaire (ICP) a été décidée. \\n\\nMme Dubois a été rapidement transférée en salle de cathétérisme cardiaque. L'angiographie coronaire a confirmé une occlusion totale de l'IVA proximale. Un ballonnet a été gonflé pour rétablir le flux sanguin, suivi de l'implantation d'un stent médicamenteux. Le flux TIMI 3 (flux normal) a été obtenu après l'angioplastie. \\n\\nAprès l'intervention, Mme Dubois a été admise en unité de soins intensifs cardiologiques. Son état s'est progressivement amélioré. La douleur thoracique a disparu et sa tension artérielle s'est stabilisée. Une échocardiographie a révélé une hypokinésie de la paroi antérieure du ventricule gauche, témoignant d'une zone infarcie. Elle a été mise sous traitement médical optimal comprenant des bêta-bloquants, un inhibiteur de l'enzyme de conversion (IEC), une statine et une double antiagrégation plaquettaire (aspirine et clopidogrel). \\n\\nPendant son séjour à l'hôpital, elle a bénéficié d'un programme de réadaptation cardiaque visant à améliorer sa condition physique et à réduire les facteurs de risque cardiovasculaires. Elle a également reçu des conseils diététiques et des informations sur la gestion de son diabète. Elle est sortie de l'hôpital une semaine plus tard, avec des recommandations de suivi régulier par son cardiologue et son médecin traitant.\",
  \"MeSH_Code\": [
    \"C14 – cardiovascular diseases\",
    \"C18 – nutritional and metabolic diseases\",
    \"C23 – pathological conditions, signs and symptoms\"
  ]
}"""),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""['C04 – neoplasms', 'C08 – respiratory tract diseases', 'C14 – cardiovascular diseases']"""),
            ],
        ),
        types.Content(
            role="model",
            parts=[
                types.Part.from_text(text="""{
  \"Cas_clinique\": \"M. Lemaire, un homme de 62 ans, ancien fumeur (30 paquets-années), s'est présenté à la consultation de pneumologie en raison d'une toux persistante depuis plusieurs mois, accompagnée d'expectorations muco-purulentes et d'une dyspnée d'effort progressive. Il a également noté une perte de poids involontaire de 5 kg au cours des trois derniers mois et des épisodes occasionnels d'hémoptysie. \\n\\nL'examen clinique a révélé une diminution du murmure vésiculaire à la base du poumon droit et la présence de râles bronchiques diffus. Une radiographie thoracique a mis en évidence une opacité hilaire droite avec un élargissement médiastinal. Une tomodensitométrie (TDM) thoracique a confirmé la présence d'une masse tumorale de 5 cm de diamètre dans le hile droit, envahissant la bronche souche droite et les ganglions médiastinaux. \\n\\nUne bronchoscopie avec biopsies a été réalisée. L'examen histopathologique a révélé un carcinome épidermoïde peu différencié. Un bilan d'extension comprenant une TDM abdominale et une scintigraphie osseuse n'a pas révélé de métastases à distance. Le patient a été classé stade IIIA (T2N2M0) selon la classification TNM du cancer du poumon. \\n\\nCompte tenu du stade localement avancé de la maladie, une approche thérapeutique multimodale a été proposée, comprenant une chimiothérapie néoadjuvante à base de platine (cisplatine et gemcitabine) suivie d'une radiothérapie conformationnelle avec modulation d'intensité (RCMI). Après trois cycles de chimiothérapie, une TDM de réévaluation a montré une réduction significative de la taille de la tumeur et des ganglions médiastinaux. Le patient a ensuite reçu une radiothérapie à la dose de 60 Gy sur le hile droit et le médiastin. \\n\\nAu cours du traitement, M. Lemaire a développé une mucite de grade 2 et une œsophagite radique nécessitant une prise en charge symptomatique avec des antalgiques et des protecteurs gastriques. Il a également présenté une pneumonie radique, traitée par corticothérapie. \\n\\nUn suivi régulier a été mis en place après la fin du traitement. Un an après, une TDM de contrôle n'a révélé aucune récidive tumorale. Cependant, M. Lemaire a développé une cardiopathie ischémique, se manifestant par des épisodes d'angor d'effort. Une coronarographie a révélé des sténoses significatives de l'artère coronaire droite et de l'artère interventriculaire antérieure. Une angioplastie coronaire avec pose de stents a été réalisée. Le patient a été mis sous traitement antiagrégant plaquettaire et bêta-bloquant. Il participe actuellement à un programme de réadaptation cardiaque et a arrêté de fumer.\",
  \"MeSH_Code\": [
    \"C04 – neoplasms\",
    \"C08 – respiratory tract diseases\",
    \"C14 – cardiovascular diseases\"
  ]
}"""),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=input_text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(
                category="HARM_CATEGORY_CIVIC_INTEGRITY",
                threshold="BLOCK_ONLY_HIGH",  # Block few
            ),
        ],
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        required = ["MeSH_Code", "Cas_clinique"],
                        properties = {
                            "MeSH_Code": genai.types.Schema(
                                type = genai.types.Type.ARRAY,
                                items = genai.types.Schema(
                                    type = genai.types.Type.STRING,
                                ),
                            ),
                            "Cas_clinique": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                        },
                    ),
    )

    full_response = ""
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        full_response += chunk.text

    return full_response


## Requests

In [39]:
rand_index = np.random.randint(0, df.shape[0])  # tire un entier aléatoire entre 0 et range_max - 1
Code_MeSH = df.loc[rand_index, "target"]

MeSH_input = mp.mesh_labels_from_vector(Code_MeSH)
MeSH_input

['C04 – neoplasms']

In [16]:
input_text = MeSH_input
chat = generate(input_text)
reponse = chat  # Puisque `generate` retourne déjà la chaîne complète

In [24]:
data = json.loads(reponse)
data['Cas_clinique']
data['MeSH_Code']

['C04 – neoplasms',
 'C08 – respiratory tract diseases',
 'C14 – cardiovascular diseases']